# CDalgs - Community detection algorithms on fsv dataset

There is a overview of detection of communities on time series. 
The dataset consists of 2D curves; each curve consists of points with the two coordinates (x, y). 
The concept is defined as follow. The shape is (batch, agents, features):
  - the batch are particular samples (curves)
  - agents are particular points of the curve
  - features are 'x' and 'y' coordinates of each point



Load libraries

In [ ]:
using CDalgs
using BSON, DataFrames
using Graphs, SimpleWeightedGraphs
using Interact, Plots

Load dataset

In [ ]:
dataset_a = BSON.load("dataset_a.bson")
data = dataset_a[:data_tensor];

Correlation graph on angents with few features & clustering function

In [ ]:
function get_clusters_ff(smoothness, clustering)
    clustering(correlation_graph_ff(data; smoothness=smoothness))
end

Evaluate clustering methods

In [ ]:
ncc95(g) = nc_clustering(g; α=.95)
ncc99(g) = nc_clustering(g; α=.99)
ncc999(g) = nc_clustering(g; α=.999)
clustering_algs = [louvain_clustering, ncc95, ncc99, ncc999]

up=0.6; down=0.3; step=0.05 
clusters_mapping = zeros(Int(round((up-down)/step+1)), length(clustering_algs), size(data)[2])
for (smoothness, i) in zip(down:step:up, 1:Int(round((up-down)/step+1)))
    clusters_vecs = zeros(length(clustering_algs), size(data)[2])
    for (clustering, j) in zip(clustering_algs, 1:length(clustering_algs))
        clusters_vecs[j, :] .= get_clusters_ff(smoothness, clustering)
    end
    clusters_mapping[i, :, :] .= clusters_vecs
end

Visualise results of clustering on all samples from the dataset

In [ ]:
plots_mat = fill(plot(), size(clusters_mapping)[1], size(clusters_mapping)[2])
for smoothness_id=1:size(clusters_mapping)[1] for clustering_alg_id=1:size(clusters_mapping)[2]
    cmap = Int.(clusters_mapping[smoothness_id, clustering_alg_id, :])
    p = plot(title=String(
            Symbol(clustering_algs[clustering_alg_id], 
                ", smoothness: ", round(up-down+step*(smoothness_id-1), digits=2)
                )))
    for i=1:100
        sample_i = data[i, :, :].*(-1)
        last_point_idx = 1
        for j in unique(cmap)
            cmap_j_idxs = findall(x->x==j, cmap)
            tmp = sample_i[union(cmap_j_idxs, last_point_idx), :]
            last_point_idx = maximum(cmap_j_idxs)
            scatter!(p, tmp[:, 1], tmp[:, 2], color=palette(:default)[j], legend=false, markersize=2.5)
        end
        scatter!(p, 
            [sample_i[argmax(sample_i[:, 2]), :][1]], 
            [sample_i[argmax(sample_i[:, 2]), :][2]], color=:black, legend=false
        )
    end
    plots_mat[smoothness_id, clustering_alg_id] = p
end end

interactive_plot_distributions = @manipulate for 
    method = widget(Dict(String.(Symbol.(clustering_algs)) .=> 1:length(clustering_algs)), label="Method selection"), 
    smoothness = widget(sort(Dict(collect(down:step:up) .=> 1:length(collect(down:step:up))), by=x->x[1]), label="Smoothness selection")
    plot(plots_mat[smoothness[], method[]])
end



In [ ]:
interactive_plot_clusters_per_sample = @manipulate throttle=1 for i=1:100, 
    method = widget(Dict(String.(Symbol.(clustering_algs)) .=> 1:length(clustering_algs)), label="Method selection"),
    smoothness = widget(sort(Dict(collect(down:step:up) .=> 1:length(collect(down:step:up))), by=x->x[1]), label="Smoothness selection")
    
    p = plot()
    sample_i = data[i, :, :].*(-1)
    cmap = Int.(clusters_mapping[smoothness[], method[], :])
    last_point_idx = 1
    for j in unique(cmap)
        cmap_j_idxs = findall(x->x==j, cmap)
        tmp = sample_i[union(cmap_j_idxs, last_point_idx), :]
        last_point_idx = maximum(cmap_j_idxs)
        scatter!(p, tmp[:, 1], tmp[:, 2], color=palette(:default)[j], legend=false, markersize=5)
    end
    plot!(p, sample_i[:, 1], sample_i[:, 2], color=:black, legend=false, ls=:dash)
    scatter!(p, 
        [sample_i[argmax(sample_i[:, 2]), :][1]], 
        [sample_i[argmax(sample_i[:, 2]), :][2]], color=:black, legend=false
    )
    p
end